<a href="https://colab.research.google.com/github/MorozovDesu/neural_network_technologies/blob/main/%D0%9B%D0%B0%D0%B1%D0%B0_3_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%B5%D1%82%D0%B5%D0%B2%D1%8B%D0%B5_%D1%82%D0%B5%D1%85%D0%BD%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ВАРИАНТ РАБОТЫ имеет вид Z-f1-f2, где
      Z=1-6 – вид закона распределения генерируемых признаков

  1.     Биномиальное

  2.     Геометрическое

  3.     Гипергеометрическое

  4.     Пуассона

  5.     Равномерное

  6.     Нормальное

f1 -вид функциональной зависимости от одного аргумента

f1=1-12

 1-квадратичная функция

 2- синусоидальная функция

 3-логарифмическая функция

 4-экспоненциальная функция

 5-корень квадратный

 6-тангенс

 7- кубическая функция

f2 - вид функциональной зависимости от двух аргументов

Функция Бирда

мой вариант z=5, f1=3, f2=8;

In [1]:
import numpy as np
from numpy.random import uniform
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [2]:
x1val = np.random.uniform(-5, 5, 500)
x2val = np.random.uniform(-4, 4, 500)

x1, x2 = np.meshgrid(x1val, x2val)
def f(x, y):
    return np.sin(x-y)
# приравниваем z к функции от x и y
z = f(x1,x2)
x11=[]
x22=[]
zz=[]
for l in x1:
    x11.extend(l)
for l in x2:
    x22.extend(l)
for l in z:
    zz.extend(l)

x11=np.reshape(x11,(len(x11),1))
x22=np.reshape(x22,(len(x22),1))
zz=np.reshape(zz,(len(zz),1))



In [3]:
dataset =np.concatenate((x11,x22),axis=1)

X_train, X_test, y_train, y_test = train_test_split(dataset, zz, test_size=0.4, random_state=123)


Сеть прямого распространения сигнала и обратного распространения ошибки

In [4]:
from random import seed
from random import random
from math import exp



In [5]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
  network = list()
  hidden_layer = [{'weights':[random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
  network.append(hidden_layer)
  output_layer = [{'weights':[random() for i in range(n_hidden +1)]} for i in range(n_outputs)]
  network.append(output_layer)
  return network
seed(1)


In [6]:
# Calculate neuron activation for an input
def activate(weights, inputs):
  activation = weights[-1]
  for i in range(len(weights)-1):
    activation += weights[i] * inputs[i]
  return activation

# Transfer neuron activation
def transfer(activation):
  sh=(round(exp(alpha*activation),5) - round(exp(-alpha*activation),5))/2
  ch=(round(exp(alpha*activation),5) + round(exp(-alpha*activation),5))/2
  return sh/ch

# Forward propagate input to a network output
def forward_propagate(network, row):
  inputs = row
  for layer in network:
    new_inputs = []
    for neuron in layer:
      activation = activate(neuron['weights'], inputs)
      neuron['output'] = transfer(activation)
      new_inputs.append(neuron['output'])
    inputs = new_inputs
  return inputs

# Calculate the derivative of an neuron output
def transfer_derivative(output):
  return 2*alpha*(output+1) * (1.0 - (output+1)/2)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
  for i in reversed(range(len(network))):
    layer = network[i]
    errors = list()
    if i != len(network)-1:
      for j in range(len(layer)):
        error = 0.0
        for neuron in network[i + 1]:
          error += (neuron['weights'][j] * neuron['delta'])
        errors.append(error)
    else:
      for j in range(len(layer)):
        neuron = layer[j]
        errors.append(expected[j] - neuron['output'])
    for j in range(len(layer)):
      neuron = layer[j]
      neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
# Update network weights with error
def update_weights(network, row, l_rate):
  for i in range(len(network)):
    inputs = row[:-1]
    if i != 0:
      inputs = [neuron['output'] for neuron in network[i - 1]]
    for neuron in network[i]:
      for j in range(len(inputs)):
        neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
      neuron['weights'][-1] += l_rate * neuron['delta']



In [7]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
  for epoch in range(n_epoch):
    sum_error = 0
    for row in train:
        outputs = forward_propagate(network, row)
        expected = [0 for i in range(n_outputs)]
        expected[0] = row[-1]
        sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
        backward_propagate_error(network, expected)
        update_weights(network, row, l_rate)
    print('&gt;epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))


In [8]:

dataset1 =np.concatenate((X_train,y_train),axis=1)
dataset1 = dataset1.tolist()
n_inputs = len(dataset1[0]) - 1
n_outputs = 1
alpha=0.2
l_rate=0.3
n_epoch=10

network = initialize_network(n_inputs,10, n_outputs)
train_network(network, dataset1, 0.5, n_epoch, n_outputs)



&gt;epoch=0, lrate=0.500, error=33167.278
&gt;epoch=1, lrate=0.500, error=21540.633
&gt;epoch=2, lrate=0.500, error=11789.768
&gt;epoch=3, lrate=0.500, error=4768.296
&gt;epoch=4, lrate=0.500, error=1820.499
&gt;epoch=5, lrate=0.500, error=1090.036
&gt;epoch=6, lrate=0.500, error=1066.947
&gt;epoch=7, lrate=0.500, error=1072.961
&gt;epoch=8, lrate=0.500, error=1074.180
&gt;epoch=9, lrate=0.500, error=1073.854


In [11]:
# # Make a prediction with a network
def predict(network, row):
  outputs = forward_propagate(network, row)
  return outputs
i=-1
for row in dataset1:
    i=i+1
    prediction = predict(network, row)
    if (i==0):
      predictTrain = np.array(prediction)
    else:
      predictTrain =np.append(predictTrain,prediction,axis=0)

print(predictTrain)

# %matplotlib notebook

# x1111 = np.linspace(-5, 5, num=100)
# x2222 = np.linspace(-4, 4, num=100)
# x111, x222 = np.meshgrid(x1111, x2222)
# y111 = f(x111, x222)
# fig = plt.figure(figsize=(16,8))
# ax = fig.add_subplot(111, projection='3d')
# col = np.arange(10000)
# ax.scatter(x111, x222, y111, marker = 'o', linewidths=1, c=col)

# ax.scatter(X_train[:, 0], X_train[:, 1], predictTrain, marker = 'o', linewidths=3,c='RED')
# ax.view_init(10, 70)



[ 0.86719981 -0.74061394  0.7589861  ...  0.68103833 -0.27564512
 -0.21815591]


In [14]:
import plotly.graph_objs as go
import numpy as np

# Define the Bird function
def bird(x):
    return np.sin(x[0]) * np.exp((1 - np.cos(x[1]))**2) + \
           np.cos(x[1]) * np.exp((1 - np.sin(x[0]))**2) + \
           (x[0] - x[1])**2

# Create a meshgrid of x and y values
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)

# Evaluate the Bird function at each point in the meshgrid
Z = bird([X, Y])

# Create a surface plot of the Bird function
fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z)])
fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))
fig.show()
